# Backpropagation

**(C) 2018 by [Damir Cavar](http://damir.cavar.me/)**

**Version:** 1.0, January 2018

**License:** [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/) ([CA BY-SA 4.0](https://creativecommons.org/licenses/by-sa/4.0/))

This is a tutorial related to the L665 course on Machine Learning for NLP focusing on Deep Learning, Spring 2018 at Indiana University.

This material is based on Matt Mazur's tutorial [A Step by Step Backpropagation Example](https://mattmazur.com/2015/03/17/a-step-by-step-backpropagation-example/). See for more details and explanations this page. Matt also provides a Python example script with the necessary code in one of [his GitHub repos](https://github.com/mattm/simple-neural-network). All copyrights are his, except on a few small comments that I added.

## Introduction

In this example we will use a very simple network to start with. The network will only have one input and one output layer. We want to make the following predictions from the input:

| Input  | Output |
| ------ |:------:|
| 0 0 1  | 0      |
| 1 1 1  | 1      |
| 1 0 1  | 1      |
| 0 1 1  | 0      |

We will use *Numpy* to compute the network parameters, weights, activation, and outputs:

In [39]:
import numpy as np

We will use the *Sigmoid* activation function:

In [53]:
def sigmoid(x, deriv=False):
    if(deriv==True):
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

The Sigmoid activation function introduces non-linearity to the computation. It maps the input value to an output value between $0$ and $1$. The sigmoid function returns the derivative of the sigmoid for any given $x$, if the second parameter *deriv* is set to *True*. The derivative of the sigmoid is $x * (1 - x)$. This is basically the  slope of the sigmoid function at any given point, see below:

<img src="SigmoidFunction1.png" style="max-width:100%; width: 30%; max-width: none">

We define the inputs as rows in *X*. There are three input nodes (three columns per vector in $X$. Each row is one trainig example:

In [57]:
X = np.array([ [ 0, 0, 1 ],
               [ 0, 1, 1 ],
               [ 1, 0, 1 ],
               [ 1, 1, 1 ] ])
print(X)

[[0 0 1]
 [0 1 1]
 [1 0 1]
 [1 1 1]]


The outputs are stored in *y*, where each row represents the output for the corresponding input vector (row) in *X*. The vector is initiated as a single row vector and with four columns and transposed (using the $.T$ method) into a column vector with four rows:

In [58]:
y = np.array([[0,0,1,1]]).T
print(y)

[[0]
 [0]
 [1]
 [1]]


To make the outputs deterministic, we seed the random number generator with a constant. This will guarantee that every time you run the code, you will get the same random distribution:

In [62]:
np.random.seed(1)

We create a weight matrix with randomly initialized weights:

In [63]:
n_inputs = 3
n_outputs = 1
syn0 = 2 * np.random.random( (n_inputs, n_outputs) ) - 1
print(syn0)

[[-0.16595599]
 [ 0.44064899]
 [-0.99977125]]


The reason for the weight matrix to have 3 rows and 1 column is that it represents the weights of the connections from the three input neurons to the single output neuron. The initialization of the weight matrix is random with a mean of zero. There is a good reason for chosing a mean of zero in the weight initialization.

The core representation of this network is basically the weight matrix *syn0*. The rest, input matrix, output vector and so on are components that we need to learning and evaluation. The leraning result is stored in the *syn0* weight matrix.

We loop in the optimization and learning cycle 10,000 times. In the *forward propagation* line we process the entire input matrix for training. This is called **full batch** training. I do not use an alternative variable name to represent the input layer, instead I use the input matrix $X$ directly here. Think of this as the different inputs to the input neurons computed at once. In principle the input or training data could have many more training examples, the code would stay the same.

In [76]:
for n in range(10000):
    # forward propagation
    l1 = sigmoid(np.dot(X, syn0))
    
    # compute the loss
    l1_error = y - l1
    #print("l1_error:\n", l1_error)
    
    # multiply the loss by the slope of the sigmoid at l1
    l1_delta = l1_error * sigmoid(l1, True)
    #print("l1_delta:\n", l1_delta)
    
    #print("error:", l1_error, "\nderivative:", sigmoid(l1, True), "\ndelta:", l1_delta, "\n", "-"*10, "\n")
    # update weights
    syn0 += np.dot(X.T, l1_delta)

print("l1:\n", l1)

l1:
 [[0.00264469]
 [0.00215725]
 [0.99823945]
 [0.99784145]]


The dots in $l1$ represent the lines in the graphic below. The lines represent the slope of the sigmoid in the particular position. The slope is highest with a value $x = 0$ (blue dot). It is rather shallow with $x = 2$ (green dot), and not so shallow and not as high with $x = -1$. All derivatives are between $0$ and $1$, of course, that is, no slope or a maximal slope of $1$. There is no negative slope in a sigmoid function.

<img src="sigmoid-deriv-2.png" style="max-width:100%; width: 50%; max-width: none">

The matrix $l1\_error$ is a 4 by 1 matrix (4 rows, 1 column). The derivative matrix $sigmoid(l1, True)$ is also a 4 by one matrix. The returned matrix of the element-wise product $l1\_delta$ is also the 4 by 1 matrix.

The product of the error and the slopes **reduces the error of high confidence predictions**. When the sigmoid slope is very shallow, the network had a very high or a very low value, that is, it was rather confident. If the network guessed something close to $x=0, y=0.5$, it was not very confident. Such predictions without confidence are updated most significantly. The other peripheral scores are multiplied with a number closer to $0$.

In the prediction line $l1 = sigmoid(np.dot(X, syn0))$ we compute the dot-product of the input vectors with the weights and compute the sigmoid on the sums.
The result of the dot-product is the number of rows of the first matrix ($X$) and the number of columns of the second matrix ($syn0$).
In the computation of the difference between the true (or gold) values in $y$ and the "guessed" values in $l1$ we have an estimate of the miss.

An example computation for the input $[ 1, 0, 1 ]$ and the weights $[ 9.5, 0.2, -0.1 ]$ and an output of $0.99$: If $y = 1$, the $l1\_error = y - l2 = 0.01$, and $l1\_delta = 0.01 * tiny\_deriv$:

<img src="toy_network_deriv.png" style="max-width:100%; width: 50%; max-width: none">

## More Complex Example